In [1]:
import matplotlib
import matplotlib.pyplot as plt

import os

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
import time
from tqdm import tqdm

from PIL import Image

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device 객체

In [2]:
device

device(type='cuda', index=0)

In [3]:
# 데이터셋을 불러올 때 사용할 변형(transformation) 객체 정의
transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 정규화(normalization)
])

In [4]:
from torch.utils.data import Dataset, DataLoader

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [5]:
test_dir = '/opt/ml/input/cropped_v2.1/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))

In [6]:
def model_run(target, sub_df, test_dir):
    data_dir = f'/opt/ml/input/cropped_v2.1/train/images_classified_{target}/'
    
    dataset = datasets.ImageFolder(os.path.join(data_dir), transforms)
    
    print(f'*****{target}*****')
    print(dataset)
    print()
    
    class_names = dataset.classes
    print(class_names)
    print()
    
    train_size = int(0.8 * len(dataset))
    valid_size = len(dataset) - train_size
    
    train_set, valid_set = torch.utils.data.random_split(dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(42))
    
    dataloader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=8)
    
    model = models.resnet34(pretrained=True)

    num_features = model.fc.in_features
    # 전이 학습(transfer learning): 모델의 출력 뉴런 수를 18개로 교체하여 마지막 레이어 다시 학습
    model.fc = nn.Linear(num_features, len(class_names)) 
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    num_epochs = 10
    model.train()
    start_time = time.time()

    # 전체 반복(epoch) 수 만큼 반복하며
    for epoch in range(num_epochs):
        running_loss = 0.
        running_corrects = 0

        # 배치 단위로 학습 데이터 불러오기
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 모델에 입력(forward)하고 결과 계산
            optimizer.zero_grad() # 전체 grad 값을 초기화.
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # 역전파를 통해 기울기(gradient) 계산 및 학습 진행
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_set)
        epoch_acc = running_corrects / len(train_set) * 100.

        # 학습 과정 중에 결과 출력
        print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))
        
    valid_dataloader = torch.utils.data.DataLoader(valid_set, batch_size=128, shuffle=False, num_workers=8)
    
    model.eval()
    start_time = time.time()

    all_labels = []
    all_preds = []

    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0

        for inputs, labels in valid_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        
            all_preds.extend(preds.cpu().numpy())
        
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        
            all_labels.extend(labels.data.cpu().numpy())
        
            '''
            # 한 배치의 첫 번째 이미지에 대하여 결과 시각화
            print(f'[예측 결과: {class_names[preds[0]]}] (실제 정답: {class_names[labels.data[0]]})')
            imshow(inputs.cpu().data[0], title='예측 결과: ' + class_names[preds[0]])
            '''
            
        epoch_loss = running_loss / len(valid_set)
        epoch_acc = running_corrects / len(valid_set) * 100.
        print('[Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch_loss, epoch_acc, time.time() - start_time))
        
    print(f'*****F1 Score: {f1_score(all_labels, all_preds, average='macro')}*****')
    
    image_dir = os.path.join(test_dir, 'images')
    
    image_paths = [os.path.join(image_dir, img_id) for img_id in sub_df.ImageID]
    
    test_set = TestDataset(image_paths, transforms)
    
    test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=8)
    
    all_predictions = []

    with torch.no_grad():
        for inputs in tqdm(test_dataloader):
            inputs = inputs.to(device)
        
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        
            all_predictions.extend(preds.cpu().numpy())
    
    all_predictions2 = []

    for p in all_predictions:
        all_predictions2.append(class_names[p])
        
    sub_df[target] = all_predictions2
    
    return sub_df

In [7]:
submission = model_run('mask', submission, test_dir).copy()

*****mask*****
Dataset ImageFolder
    Number of datapoints: 18900
    Root location: /opt/ml/input/cropped_v2.1/train/images_classified_mask/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

['0', '12', '6']



  0%|          | 0/119 [00:00<?, ?it/s]

#0 Loss: 0.1814 Acc: 93.7302% Time: 27.1434s


  0%|          | 0/119 [00:00<?, ?it/s]

#1 Loss: 0.0220 Acc: 99.4180% Time: 54.0785s


...

100%|██████████| 119/119 [00:26<00:00,  4.44it/s]

#9 Loss: 0.0040 Acc: 99.9206% Time: 268.4743s



  0%|          | 0/99 [00:00<?, ?it/s]

[Test Phase] Loss: 0.0088 Acc: 99.7090% Time: 3.3496s


100%|██████████| 99/99 [00:09<00:00, 10.83it/s]


In [8]:
submission = model_run('gender', submission, test_dir).copy()

*****gender*****
Dataset ImageFolder
    Number of datapoints: 18900
    Root location: /opt/ml/input/cropped_v2.1/train/images_classified_gender/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

['0', '3']



  0%|          | 0/119 [00:00<?, ?it/s]

#0 Loss: 0.2011 Acc: 91.5873% Time: 26.9281s


  0%|          | 0/119 [00:00<?, ?it/s]

#1 Loss: 0.0464 Acc: 98.6177% Time: 53.6633s


...

100%|██████████| 119/119 [00:26<00:00,  4.44it/s]

#9 Loss: 0.0020 Acc: 100.0000% Time: 268.1477s



  0%|          | 0/99 [00:00<?, ?it/s]

[Test Phase] Loss: 0.0131 Acc: 99.5503% Time: 3.4691s


100%|██████████| 99/99 [00:09<00:00, 10.81it/s]


In [9]:
submission = model_run('age', submission, test_dir).copy()

*****age*****
Dataset ImageFolder
    Number of datapoints: 18900
    Root location: /opt/ml/input/cropped_v2.1/train/images_classified_age/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

['0', '1', '2']



  0%|          | 0/119 [00:00<?, ?it/s]

#0 Loss: 0.3961 Acc: 85.5225% Time: 26.7274s


  0%|          | 0/119 [00:00<?, ?it/s]

#1 Loss: 0.1932 Acc: 92.5066% Time: 53.5822s


...

100%|██████████| 119/119 [00:26<00:00,  4.45it/s]

#9 Loss: 0.0051 Acc: 99.9868% Time: 267.4753s



  0%|          | 0/99 [00:00<?, ?it/s]

[Test Phase] Loss: 0.0832 Acc: 96.9841% Time: 3.2977s


100%|██████████| 99/99 [00:09<00:00, 10.85it/s]


In [10]:
submission = submission.astype({'age':'int','gender':'int','mask':'int'})

In [11]:
try : submission['ans'] = submission['mask'] + submission['gender'] + submission['age'] ; submission2 = submission.drop(['mask','gender','age'],axis=1)
except : pass
submission2

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,2
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [12]:
submission2.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)